In [27]:
import json
import urllib

import metpy
import pandas as pd
import pint

units = pint.UnitRegistry()

with urllib.request.urlopen('http://xmountwashington.appspot.com/mmNew.php?callback=null') as response:
    callback = response.read()
json_string = callback.decode("utf8").rstrip(')').lstrip('null(')
mesonet_data = json.loads(json_string)
print(mesonet_data['currentMesonetConditions']['AR16'])

def make_item(json):
    '''
    Takes a json object and creates an items' dataframe
    '''
    df = pd.DataFrame(columns=["siteName", "siteAltitudeM", "temperature"])
    for site in json:
        if "temperature" in json[site]:
            df = df.append({"siteName": site, 
                            "siteAltitude": 999 * units.meters, 
                            "temperature": json[site]["temperature"] * units.degF}, 
                           ignore_index=True)
    return df

def get_virtual_temperature(df):
    for row in df:
        if "pressure" not in row:
            pressure = height_to_standard_pressure(row["siteAltitude"])
        else:
            pressure = row["pressure"]
            
        if "dewpoint" in row:
            # calculate equivalent potential temperature
            metpy.calc.equivalent_potential_temperature(pressure, row["temperature"], row["dewpoint"])
        else:
            # calculate potential temperature
            metpy.calc.potential_temperature(pressure, row["temperature"])
            
        metpy.calc.virtual_temperature(row["temperature"], mixing, molecular_weight_ratio=<Quantity(0.6219800858985514, 'dimensionless')>)

def height_to_pressure(altitude, df):
    standard_pressure = metpy.calc.height_to_pressure_std(altitude)
    if "pressure" in df.columns:
        # correct standard pressure with what we have in the dataframe
    return standard_pressure
    

dataframe = make_item(mesonet_data['currentMesonetConditions'])
print(dataframe)
            
# Should be as simple as pd.read_json once we figure out the callback issue
#pd.read_json('http://xmountwashington.appspot.com/mmNew.php')

{'experation': '2019-02-08T04:30:00+00:00', 'temperature': '31.0', 'humidity': '93', 'wind': 'S 12.6', 'pressure': None, 'latitude': 44.290001, 'longitude': -71.226652}
   siteName siteAltitudeM temperature
0      AR16           999        31.0
1      AR23           999        27.7
2      AR33           999        32.6
3      AR40           999        37.0
4      AR43           999        36.1
5      AR53           999        33.0
6      ATTI           999        27.4
7      BRWD           999        30.2
8      CANN           999        28.8
9      CRAN           999        28.6
10     HICR           999        29.0
11     JACK           999        27.4
12     MIZP           999        26.3
13     SUMT           999        30.1
14     TUCK           999        37.2
15     WILD           999        22.0
16     NCON           999        32.0
   siteName siteAltitudeM temperature
0      AR16           999        31.0
1      AR23           999        27.7
2      AR33           999        